# Micro-452 - Basics of Mobile Robotics : Final Project

##### Group n. 14. Members : Enrique Bosch Tamayo, Edoardo Finamore, Aubane Lachat, Louis Litzler-Italia

***Our project is divided into two parts. This Jupyter Notebook comprises the theoretical report, explaining the implementation, our choices, the calculations, and includes some pieces of code to elucidate tricky aspects. To execute the entire code and visualize the results, you'll need to open the second Jupyter Notebook named 'main.ipynb' (which is entirely separate from this one). There, you'll find the main code and the plotting code to visualize the robot's path, its position...***

- [1.Introduction](#1.Introduction)
- [2.Strategy](#2.Strategy)
- [3.Implementation](#3.Implementation)
    - [3.1Setup of the environment](#3.1Setup-of-the-environment)
    - [3.2Create an occupancy grid using the camera](#3.2Create-an-occupancy-grid-using-the-camera)
    - [3.2.1Main-parameters-initialization](#3.2.1Main-parameters-initialization)
        - [3.2.2Corner position extraction](#3.2.2Corner-position-extraction)
        - [3.2.3Reshape and resize the image](#3.2.3Reshape-and-resize-the-image)
        - [3.2.4Apply some filters](#3.2.4Apply-some-filters)
        - [3.2.5Grow the obstacles](#3.2.5Grow-the-obstacles)
        - [3.2.6Extract the occupancy grid](#3.2.6Extract-the-occupancy-grid)
    - [3.3Global path planning](#3.3Global-path-planning)
        - [3.3.1Robot and goal pose](#3.3.1Robot-and-goal-pose)
        - [3.3.2A* algorithm](#3.3.2A*-algorithm)
    - [3.4Kalman filter](#3.4Kalman-filter)
        - [3.4.1Covariance matrix Q estimation](#3.4.1Covariance-matrix-Q-estimation)
        - [3.4.2Measurement model](#3.4.2Measurement-model)
        - [3.4.3Covariance R matrix estimation](#3.4.3Covariance-R-matrix-estimation)
        - [3.4.4Steps of Kalman filter](#3.4.4Steps-of-Kalman-filter)
        - [3.4.5Conversion functions from Thymio speed to \[rad/s\] and viceversa](#3.4.5Conversion-functions-from-Thymio-speed-to-[rad/s]-and-viceversa)
        - [3.4.6Transformation of coordinates used by the A* algorithm, Vision and Motion Control](#3.4.6Transformation-of-coordinates-used-by-the-A*-algorithm,-Vision-and-Motion-Control)
    - [3.5Local naviaton](#3.5Local-naviaton)
    - [3.6Control of the robot](#3.6Control-of-the-robot)
    - [3.7Kidnapping of the robot](#3.7Kidnapping-of-the-robot)
- [4.Further improvements](#4.Further-improvements)
- [Conclusions](#Conclusions)
- [References](#References)
      
    
    
    

<a id="1.Introduction"></a>
# 1.Introduction

The aim of this project is to move a Thymio robot from an arbitrary initial position to a specified goal on a map that includes static obstacles. The Thymio must also be capable of avoiding unforeseen obstacles using its sensors. Static obstacles should be avoided without relying on the robot sensors. The project is done by combining computer vision, global navigation, filtering, local navigation and motion control.  

We decided to choose an environment with a homogeneous blue background and black polygons as obstacles. This choice is explained later in the report. A typical example of such a environment is shown below : 

<div>
<center><img src="Images/typical_map_example.png" width="600"/>
</div>

A demonstration of the final result can be seen in the video below :
https://www.youtube.com/watch?v=VRTQnFyThes

<a id="2.Strategy"></a>
# 2.Strategy

Our strategy to solve the overall problem is explained in this section. The steps of our implementation are the following : 


- **Step 1** : Set the environment by adding polygon obstacles on the blue environmnent.

- **Step 2** : Extract a rasterized picture of the environment and convert it into an occupancy grid using the camera and computer vision. 

- **Step 3** : Put the robot and the goal in the environment.

- **Step 4** : Compute the optimal path from robot to goal

- **Step 5** : Once the optiaml path is computed, the Kalman filter is used to move the robot towards the goal.

- **Step 6** : If an unexpected obstacle is detected, it is avoided with local navigation. The robot reaches the optimal path again and we go back to step 5.

- **Step 7** : Find the closest temporary goal from the optimal path

- **Step 8** : Calculate the control law

- **Step 9** : If goal is reached, the robot stops. Otherwise, we go back to step 5.

If the robot is kidnapped, it stops moving. We go back at step 4 once the robot is placed somewhere in the environment. 


The finite state machine that we implemented is shown below : 

<div>
<center><img src="Images/strategy_scheme.jpg" width="900"/>
</div>

All these steps are explained and justified in more details in the corresponding sections. 

<a id="3.Implementation"></a>
# 3.Implementation

In this section, we explain the different steps of our implementation, we explain the choices we made, their pros and cons, and we break down our code step by step. The lines of code that are shown in this notebook cannot be run, they are here only to explain the code you can find in the main file. 

<a id="3.1Setup-of-the-environment"></a>
### 3.1Setup of the environment

To setup the environment, we have to put black polygons in a light blue background. At each corner of the map, we add an Aruco marker that will be used for the computer vision steps.  We also connect the camera and the thymio to the computer.

<div>
<center><img src="Images/setup_environment.jpg" width="600"/>
</div>

<a id="3.2Create-an-occupancy-grid-using-the-camera"></a>
### 3.2Create an occupancy grid using the camera

For this step of the implementation, the thymio and the goal should not be put in the environment. If this condition is not satisfied, the program will wait until none of these two objects are detected. 

<a id="3.2.1Main-parameters-initialization"></a>
#### 3.2.1Main parameters initialization

Once the environment is setup, we define the constants that are necessary for our implementation.

In [ ]:
############################## MAIN PARAMETERS INITIALIZATION ###################################

# Initialisation

# Initial state estimate
state_estimate_k_minus_1 = np.array([0.0,0.0,0.0])

# Sample time
Ts = 0.2

# Convertion ratios for speed calculations
conv_ratio_left = 59
conv_ratio_right = 59

# Wheels parameters
R = 20 #mm
L = 105 #mm

# State covariance matrix P_k_minus_1
P_k_minus_1 = np.array([[0.1,0,0],
                    [0,0.1,0],
                    [0,0,0.1]]) 

# Initialize control vector
control_vector_k_minus_1 = [0,0]
k_step_goal = 0 # Temporary goal index

# Initialize boolean variables
not_converged = 1
camera_is_obstructed = 0
adaptable_speed = 0

# Treshold for convergence
trsh = 50 # [mm]
k_step_goal = 1 # Temporary goal index

# Map size in mm
map_size=(740,1120)

<a id="3.2.2Corner-position-extraction"></a>
#### 3.2.2Corner position extraction

Then, we open the camera and we extract the corner postions using the function 'detect_map_corners', defined in the file vision.py.

In [ ]:
corner_position, last_frame = detect_map_corners(camera_index=0) # Returns the position of the corners of the map as well as an image of the map

<div>
<img src="Images/last_frame_detect_corner.png" width="600"/>
</div>

Using the image and the provided corner positions, we call the function 'img_to_grid' to generate the occupancy grid. This function, defined in the file vision.py, performs various computer vision steps to achieve its goal. 

<a id="3.2.3Reshape-and-resize-the-image"></a>
#### 3.2.3Reshape and resize the image

The saved frame is reshaped and resized in order to be coherent with the proportional to the shape of the real environment.  

In [ ]:
reshaped_img = reshape_img(last_frame, map_size, corner_position)

<div>
<img src="Images/resized_img.png" width="600"/>
</div>

<a id="3.2.4Apply-some-filters"></a>
#### 3.2.4Apply some filters

We apply a bilateral filter to the image, with kernel size 9. The objective is to get rid of noise while accentuating the borders of objects we detect. We do this because we will later on apply a contour detection function to identify the obstacles on the map. (Other more computationally light filters may be applied for similar results but a bilateral filter works best for our situation and we do not need to change it yet for performances’ sake).

We then apply a gray filter to the image with “cv2.cvtColor()”, which we then convert into a binary image with cv2.threshold(). We have chosen a threshold of [120,200] pixel intensity which works just fine to isolate the polygonal obstacles.

In [ ]:
# Return occupancy_grid from and image
bilateral = cv2.bilateralFilter(reshaped_img,9,75,75)
bw_bilateral = cv2.cvtColor(bilateral, cv2.COLOR_BGR2GRAY)

ret,binary_image = cv2.threshold(bw_bilateral,120, 200,cv2.THRESH_BINARY)

<div>
<img src="Images/binary_img.png" width="600"/>
</div>

<a id="3.2.5Grow-the-obstacles"></a>
#### 3.2.5Grow the obstacles

Since our program considers the robot to be a single point, we need to grow the obstacles to avoid hitting their edges with the thymio. Another reason to grow the obstacles is the overshoot observed during the control of the robot, which leads the thymio not to always follow precisely the optimal path. We use the function cv2.erode() to achieve this task. It does not keep the exact same obstacle shapes, but we played with the number of iterations to get a good security margin. 

In [ ]:
# Grow the obstacles
kernel = np.ones((5, 5), np.uint8)
dilated_image = cv2.erode(binary_image, kernel, iterations=15) #increase nb of iterations to increase dilation

<div>
<img src="Images/grow_obstacles.png" width="600"/>
</div>

<a id="3.2.6Extract-the-occupancy-grid"></a>
#### 3.2.6Extract the occupancy grid

Finally, we discretize the image so we can extract an occupancy grid out of it. We first have to resize the image so that whatever cell size we choose from, there will be no remainder after we have done so ( for example, if our map is 720 x 1080 pixels, then splitting it into 17 pixel cells will leave 6 pixels as remainder on the x axis, so we’ll resize the image to 714 x 1071). We then check for each cell area in the resulting image, if the majority of pixels are black or white, and assign to that cell a boolean determined by the majority winner. As a result, we get a boolean list of cells or occupancy grid of the state of the map, where each cell is assigned a ‘0’ if the cell is occupied by an obstacle, and ‘1’ if the cell is free.

In [ ]:
cell_size = 20  # Adjust the cell size as needed
height, width = dilated_image.shape

new_height = (height // cell_size) * cell_size
new_width = (width // cell_size) * cell_size
resized_result_image = cv2.resize(dilated_image, (new_width, new_height))

occupancy_grid = np.zeros((new_height // cell_size, new_width // cell_size), dtype=np.uint8)

for i in range(0, new_height, cell_size):
        for j in range(0, new_width, cell_size):
            cell = resized_result_image[i:i+cell_size, j:j+cell_size]
            occupancy = np.sum(cell == 0) / (cell_size * cell_size)
            occupancy_grid[i // cell_size, j // cell_size] = occupancy > 0.5

<div>
<img src="Images/occupancy_grid_image.png" width="400"/>
</div>

We can play with the number of iteration in the growing obstacles step, as well as with the cell size defined just above, to get a more or less precise representation of the real environment. We decided to choose a number of iterations to grow the obstacles equal to 20 and a cell size equal to 20. It was a good trade off between a good representation of the real environment, and a good security margin that takes into account the robot size and the control overshoot. We also had to take into account the fact that a small cell size results in a way larger computational cost (for example the optimal path will take way more time to be computed). 

Below, you can see the results for other choices of parameters. The first image isn't grown and has a cell size of 1. It is the most precise representation of the environment we can get using a grid. The second one has a number of iterations set to 10 and a cell size set to 8.

![Image 1](Images/size_1_grown_0.png) ![Image 2](Images/size_10_grown_8.png)

<a id="3.3Global-path-planning"></a>
### 3.3Global path planning

We decided to implement an A* algorithm for its various advantages : 

- It is a complete algorithm, which means that if a solution does exist, the algorithm will always find it 
- It is more efficient than a classic Dijkstra's algorithm
- It is straightforward to compute and it is well adapted to a matrix-type occupancy grid

Its major drawback is that it is computationnaly heavy for high dimensional grids.

To compute the optimal path, we need a start point and a goal point. 

<a id="3.3.1Robot-and-goal-pose"></a>
#### 3.3.1Robot and goal pose

Once we have the occupancy grid, we put the robot and the goal in an arbitrary position on the map and we detect them using the aruco markers. The camera will save a frame only if the robot and the goal are detected on the same frame. If the condition is not satisfied, the program will get new frames from the camera until it is satisfied. 

<div>
<center><img src="Images/robot_and_goal_pos.jpg" width="600"/>
</div>

In [ ]:
####################### START AND GOAL ############################

# Capture the initial frame for initial robot and goal poses
frame_robot_goal, cap = capture_frame(camera_index=1) # Returns a frame only if the robot and the goal markers are detected at the same time

# Get the goal position in mm in the image coordinate system
goal_pos_mm = get_goal_pos(frame_robot_goal, map_size, corner_position) 

# Get the initial pose of the robot in mm in the image coordinate system
state_vision_mm = np.array([0, 0, 0])
state_vision_mm = get_robot_pose(frame_robot_goal, map_size, corner_position, state_vision_mm)

# Get the initial pose of the robot in mm in the control coordinate system
state_vision_control  = img_to_map_coordinates(state_vision_mm, map_size[0])
optimal_state_estimate_k = state_vision_control

# Get the initial pose of the robot in grid cells in the grid coordinate system
square_size = grid_to_mm(occupancy_grid, map_size) # Length of a cell of the occupancy grid

state_vision_square = (int(state_vision_mm[0] / square_size), int(state_vision_mm[1] /square_size), state_vision_mm[2])  # Robot pose in grid cells
goal_pos_square = (int(goal_pos_mm[0] / square_size), int(goal_pos_mm[1] /square_size)) # Goal pose in grid cells

state_vision_grid = (state_vision_square[1], state_vision_square[0]) # Robot pose in the grid coordinate system
goal_pos_grid = (goal_pos_square[1], goal_pos_square[0]) # Goal pose in the grid coordinate system

These lines of code detect the robot and the goal, and returns their pose.

<a id="3.3.2A*-algorithm"></a>
#### 3.3.2A* algorithm

The A* algorithm we used is the same as the one we used during the exercice session. We used the same heuristic function. It returns the optimal path as alist of tuples. We chose to allow the robot to go in all 8 directions to get the smallest possible path. 

In [ ]:
####################### A* ############################

# Run the A* algorithm to find the optimal path, returned in "gloabal navigation"
global_navigation = A_Star(state_vision_grid, goal_pos_grid, occupancy_grid, movement_type="8N")

# Multiply global navigation by cell size to transform it in mm
global_navigation = np.array(global_navigation) * square_size

# Transform the optimal_path from grid coordinate system to control coordinate system
global_navigation = grid_to_map_coordinates(global_navigation, map_size[0]) 

<div>
<img src="Images/path.png" width="400"/>
</div>

<a id="3.4Kalman-filter"></a>
### 3.4Kalman filter

The following approach for Extended Kalman filter is taken from https://automaticaddison.com/how-to-derive-the-state-space-model-for-a-mobile-robot/, one of largest robotics educational blogs online. A modification to the model described in the documentation is realized so to consider an augmented state, which allows to take into account both measurements from camera and from sensors.

In order to localize the robot in a given environment, it's possible to use sensors measurements from motors and from a camera extracting the pose of the robot. However, all sensor measurement are affected by noise. Kalman filter allows to fuse informations from several sensors by combining their statistics and considering the mathematical model of the robot motion. The Kalman filter is aimed to minimize the error between the predicted state and the measured state. 

The mathematical model of the robot motion corresponds to that of a differential drive robot. In the following, the state of the robot is defined as 5 states, [x,y,orientation, linear velocity, angular velocity], where x, and y are in mm, orientation is in rad and the linear (mm/s) and rotational (rad/s) velocities. The motion model is given by:

$$
x_{t} =
\begin{bmatrix}
x_{t} \\
y_{t} \\
\gamma_{t} \\
v_{t} \\
\omega_{t}
\end{bmatrix}
=
\begin{bmatrix}
x_{t-1} + v_{t-1}\cos (\gamma_{t-1}) \cdot \Delta t \\
y_{t-1} + v_{t-1}\sin (\gamma_{t-1}) \cdot \Delta t \\
\gamma_{t-1} + \omega_{t-1} \cdot \Delta t \\
v_{t-1} \\
\omega_{t-1}
\end{bmatrix}
=
\begin{bmatrix}
f_1 \\
f_2 \\
f_3 \\
f_4 \\
f_5
\end{bmatrix}
=
f(x_{t-1}, u_{t-1})
$$

where $u_{t-1} = [v_{t-1}, \omega_{t-1}]$ is the control input, and $\Delta t$ is the time step.

The above equations can be rewritten in the following way:

$$
x_{t} =
A_{t-1} \cdot x_{t-1} + B_{t-1} \cdot u_{t-1}
$$

Where 

$$
A_{t-1} 
= 
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}
$$

$$
B_{t-1} 
=
\begin{bmatrix}
\cos(\gamma_{t-1}) \cdot \Delta t & 0 \\
\sin(\gamma_{t-1}) \cdot \Delta t & 0 \\
0 & \Delta t\\
1 & 0 \\
0 & 1 \\
\end{bmatrix}
$$

Since the motion model is non linear, an Extended Kalman Filter is considered. Indeed it enables the Kalman Filter to be applied to systems that have nonlinear dynamics, like in the case of Thymio. In order to do that the Jacobian of the motion model is computed. The Jacobian is the matrix of all its first-order partial derivatives. The Jacobian is used to linearize the motion model around the current state estimate. The Jacobian of f1,f2, f3, f4, f5 with respect to the state is computed as follows:


$$
J_{t-1}
=
\frac{\partial f(x_{t-1}, u_{t-1})}{\partial x_{t-1}}
=
\begin{bmatrix}
\frac{\partial f_1}{\partial x_{t-1}} & \frac{\partial f_1}{\partial y_{t-1}} & \frac{\partial f_1}{\partial \gamma_{t-1}} & \frac{\partial f_1}{\partial v_{t-1}} & \frac{\partial f_1}{\partial \omega_{t-1}} \\
\frac{\partial f_2}{\partial x_{t-1}} & \frac{\partial f_2}{\partial y_{t-1}} & \frac{\partial f_2}{\partial \gamma_{t-1}} & \frac{\partial f_2}{\partial v_{t-1}} & \frac{\partial f_2}{\partial \omega_{t-1}} \\
\frac{\partial f_3}{\partial x_{t-1}} & \frac{\partial f_3}{\partial y_{t-1}} & \frac{\partial f_3}{\partial \gamma_{t-1}} & \frac{\partial f_3}{\partial v_{t-1}} & \frac{\partial f_3}{\partial \omega_{t-1}} \\
\frac{\partial f_4}{\partial x_{t-1}} & \frac{\partial f_4}{\partial y_{t-1}} & \frac{\partial f_4}{\partial \gamma_{t-1}} & \frac{\partial f_4}{\partial v_{t-1}} & \frac{\partial f_4}{\partial \omega_{t-1}} \\
\frac{\partial f_5}{\partial x_{t-1}} & \frac{\partial f_5}{\partial y_{t-1}} & \frac{\partial f_5}{\partial \gamma_{t-1}} & \frac{\partial f_5}{\partial v_{t-1}} & \frac{\partial f_5}{\partial \omega_{t-1}} \\
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & -v_{t-1}\sin(\gamma_{t-1} ) \cdot \Delta t & \cos(\gamma_{t-1} ) \cdot \Delta t & 0\\
0 & 1 & v_{t-1}\cos(\gamma_{t-1} ) \cdot \Delta t & \sin(\gamma_{t-1} ) \cdot \Delta t & 0 \\
0 & 0 & 1 & 0 & \Delta t \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}
$$



        

Considering also the process noise  $ w_{t} $ 
 is also considered, the state space model can be then defined as:
$$x_{t}= A_{t-1}x_{t-1} + B_{t-1}u_{t-1} +w_{t-1}$$

where $ w_{t} = [ w_{t,1}, w_{t,2}, w_{t,3}, w_{t,4}, w_{t,5} ] $

While cosidering the nonlinear behaviour of the system, the KF is no more able to minimize the variance of the error between the filtered state and the real state. However, the EKF is considered in literature as a very good filter for nonlinear systems, with the ability to fuse different measurement sources. 

In the following, we consider the noise on pose of the robot to follow a Normal distribution with zero mean and covariance matrix Q (assumed to be time invariant), whose calculation is described in the following paragraph.


<a id="3.4.1Covariance-matrix-Q-estimation"></a>
#### 3.4.1Covariance matrix Q estimation

Given 5-dimensional state $ [x, y, \gamma, v, \omega] $, the corresponding Q covariance matrix for the process noise has the following block diagonal structure:



$$ Q = \begin{bmatrix}
Q_{states} & 0 \\
0 & Q_{velocities} 
\end{bmatrix} $$

where $Q_{states}$ is the covariance matrix for the first 3 states and $Q_{velocities}$ is the covariance matrix for the states v and $\omega$.

The covariance matrix accounts for unknown disturbances and model mismatch. Usually is found by trial and error, but an initial estimation can be done by considering the procedure described in the following.

The covariance matrix for the camera measurements is computed as follows:

$$ 
Q_{states} = \begin{bmatrix}
\text{Var}(x) & \text{Cov}(x,y) & \text{Cov}(x,\gamma) \\
\text{Cov}(y,x) & \text{Var}(y) & \text{Cov}(y,\gamma) \\
\text{Cov}(\gamma,x) & \text{Cov}(\gamma,y) & \text{Var}(\gamma)
\end{bmatrix} $$

The covariance matrix for the sensors measurements is computed as follows:

$$ Q_{velocities} = \begin{bmatrix}
\text{Var}(v) & \text{Cov}(v,\omega) \\
\text{Cov}(\omega,v) & \text{Var}(\omega)
\end{bmatrix} $$

$Q_{velocities}$ can be calculated by doing several experiments and computing the variance and covariance of the velocities. 
Different measurements of the left and right wheels velocities are taken by knowing the nominal speed. It's reasonable to assume that the left and right velocities are not correlated, hence the covariance is zero. The variance is computed as the average of the squared differences between the nominal speed and the measured speed.

The results of the experimentation are shown in the figure below.

<div>
<img src="Images/motor_measurements.png" width="600"/>
</div>


$Var(v_r) =  0.0059120551698918725 [(mm/s)^2]$

$Var(v_l) = 0.016143423250267915 [(mm/s)^2]$

It could be shown that errors on $v_{l}$ and $v_{r}$ follow a normal distribution with zero mean and variance $Var(v_{l})$ and $Var(v_{r})$ respectively.
Since all states are a linear combinations of $v_{l}$ and $v_{r}$, their error also follow a normal distribution. 

From the ralations of conversion between left and right velocities, the variances of linear and angular velocities are computed as follows:

$Var(v) = 0.00339 [(mm/s)^2]$
$Var(w) = 0.00157 [(rad/s)^2]$

It's a reasonable assumption of the control theory that the linear and angular velocities are not correlated, hence the covariance is zero. In this way the covariance matrix of the process noise is diagonal.

Knowing the relations between the states and the inputs \[$v$, $\omega$\], (B matrix):

$$x_t = x_{t-1} + \frac{R}{2} \cos(\gamma_{t-1})T_s (v_r + v_l) = f_1$$

$$y_t = y_{t-1} + \frac{R}{2} \sin(\gamma_{t-1})T_s (v_r + v_l) = f_2$$

$$\gamma_{t} = \gamma_{t-1} + \frac{R}{L} T_s (v_r - v_l) = f_3$$

we can derive the relation between the state variables errors and the input variable errors, using the error propagation formula. 
The variance of the input variables is computed empirically above. 

We can assume that $v_r$ and $v_l$ are uncorrelated, i.e. $Cov(v_l,v_r) = 0$: 

$$ Var(x) = \left( \frac{\partial f_1}{\partial v_r} \right)^2 Var(v_r) + \left( \frac{\partial f_1}{\partial v_l} \right)^2 Var(v_l) + 2 \left( \frac{\partial f_1}{\partial v_r} \right) \left( \frac{\partial f_1}{\partial v_l} \right) \text{Cov}(v_r, v_l) $$

$$ Var(y) = \left( \frac{\partial f_2}{\partial v_r} \right)^2 Var(v_r) + \left( \frac{\partial f_2}{\partial v_l} \right)^2 Var(v_l) + 2 \left( \frac{\partial f_2}{\partial v_r} \right) \left( \frac{\partial f_2}{\partial v_l} \right) \text{Cov}(v_r, v_l) $$

$$ Var(\gamma) = \left( \frac{\partial f_3}{\partial v_r} \right)^2 Var(v_r) + \left( \frac{\partial f_3}{\partial v_l} \right)^2 Var(v_l) + 2 \left( \frac{\partial f_3}{\partial v_r} \right) \left( \frac{\partial f_3}{\partial v_l} \right) \text{Cov}(v_r, v_l) $$

$$ Cov(x,y) = \left( \frac{\partial f_1}{\partial v_r} \right)\left( \frac{\partial f_2}{\partial v_r} \right) Var(v_r) + \left( \frac{\partial f_1}{\partial v_l} \right)\left( \frac{\partial f_2}{\partial v_l} \right) Var(v_l) + \left( \frac{\partial f_1}{\partial v_r} \right) \left( \frac{\partial f_2}{\partial v_l} \right) \text{Cov}(v_r, v_l) + \left( \frac{\partial f_1}{\partial v_l} \right) \left( \frac{\partial f_2}{\partial v_r} \right) \text{Cov}(v_r, v_l) $$

$$ Cov(x,\gamma) = \left( \frac{\partial f_1}{\partial v_r} \right)\left( \frac{\partial f_3}{\partial v_r} \right) Var(v_r) + \left( \frac{\partial f_1}{\partial v_l} \right)\left( \frac{\partial f_3}{\partial v_l} \right) Var(v_l) + \left( \frac{\partial f_1}{\partial v_r} \right) \left( \frac{\partial f_3}{\partial v_l} \right) \text{Cov}(v_r, v_l) + \left( \frac{\partial f_1}{\partial v_l} \right) \left( \frac{\partial f_3}{\partial v_r} \right) \text{Cov}(v_r, v_l) $$

$$ Cov(y,\gamma) = \left( \frac{\partial f_2}{\partial v_r} \right)\left( \frac{\partial f_3}{\partial v_r} \right) Var(v_r) + \left( \frac{\partial f_2}{\partial v_l} \right)\left( \frac{\partial f_3}{\partial v_l} \right) Var(v_l) + \left( \frac{\partial f_2}{\partial v_r} \right) \left( \frac{\partial f_3}{\partial v_l} \right) \text{Cov}(v_r, v_l) + \left( \frac{\partial f_2}{\partial v_l} \right) \left( \frac{\partial f_3}{\partial v_r} \right) \text{Cov}(v_r, v_l) $$

Which gives each entry of the $Q$ matrix:

$$Var(x) =  (\frac{R}{2} \cos(\gamma) T_s)^2 (Var(v_r)+Var(v_l))$$ 
$$Var(y) = (\frac{R}{2} \sin(\gamma) T_s)^2 (Var(v_r)+Var(v_l))$$ 
$$Var(\gamma) =  (\frac{R}{L}T_s)^2 (Var(v_r)+Var(v_l))$$


$$Cov(x,y) = (\frac{RT_s}{2L})^2(cos(\gamma)sin(\gamma))(Var(v_r)+var(v_l))$$
$$Cov(x,\gamma) = \frac{(R T_s)^2}{2L} cos(\gamma)(Var(v_r)-Var(v_l))$$
$$Cov(y,\gamma) = \frac{(R T_s)^2}{2L}sin(\gamma)(Var(v_r)-Var(v_l))$$

From te above equations, the B matrix depends on the state $\gamma$. Below is considered the worst case for computing $Q$, for which Var(x), Var(y) and Var(Gamma) are maximum.
By considering the behaviour of entries of B as function of gamma, we can see that the only value different from zero are the one on the diagonal.

<div>
<img src="Images/Q_estimation.png" width="600"/>
</div>


As a result we obtain the following matrix:
$$
Q = \begin{bmatrix}
        0.35289 [(mm)^2] & 0.0 & 0.0 & 0.0 & 0.0 \\
        0.0 & 0.35289 [(mm)^2]& 0.0 & 0.0 & 0.0 \\
        0.0 & 0.0 & 0.0013 [(rad)^2]& 0.0 & 0.0 \\
        0.0 & 0.0 & 0.0 & 0.00339 [(mm/s)^2] & 0.0 \\
        0.0 & 0.0 & 0.0 & 0.0 & 0.00157 [(rad/s)^2]
    \end{bmatrix}

    $$

<a id="3.4.2Measurement-model"></a>
#### 3.4.2Measurement model 
For this project, we consider both measurements coming from the camera and from the motor sensors. The measurement model is given by:

$$
z_{t} =

\begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}
\cdot
\begin{bmatrix}
x_{t-1, camera} \\
y_{t-1, camera} \\
\gamma_{t-1, camera} \\
v_{t-1, motor sensors} \\
\omega_{t-1, motor sensors}
\end{bmatrix}
$$

Considering also the measurement noise  $ v_{t} $ 
 
$$z_{t}= H_{t}x_{t} + v_{t}$$
    
where $ v_{t} = [ v_{t,1}, v_{t,2}, v_{t,3}, v_{t,4}, v_{t,5} ] $

In the following, we consider the measurements noise to follow a Normal distribution with zero mean and covariance matrix R (assumed to be time invariant), whose calculation is described in the following paragraph.

<a id="3.4.3Covariance-R-matrix-estimation"></a>
#### 3.4.3Covariance R matrix estimation

The covariance matrix R has a block diagonal structure:

$$ R = \begin{bmatrix}
R_{camera} & 0 \\
0 & R_{motor sensors}
\end{bmatrix} $$

where $R_{camera}$ is the covariance matrix for the measurement noise and $R_{motor sensors}$ is the covariance matrix for the motor sensors.

The covariance matrix $R_{camera}$ represents how good the vision algorithm is in estimating the position of the robot. It can be computed by doing several experiments with the vision algorithm: putting the robot in different positions and computing the variance of the error between the estimated position and the real position. 

The covariance matrix $R_{motor sensors}$ represents how good the motor sensors are in estimating the velocity of the robot. It can be computed by doing several experiments with the motor sensors as described above for the estimation of the $Q_{velocities}$.

**Camera obstruction:**

When the measurement from the camera are not available (i.e. the camera is obstructed), one can consider the covariance matrix $R_{camera}$ to be very high, so that the Kalman filter will rely only on the measurements from the motor sensors.

<a id="3.4.4Steps-of-Kalman-filter"></a>
#### 3.4.4Steps of Kalman filter

Kalman filter is composed of two steps: prediction and update. The prediction step is used to predict the state of the robot and the process covariance matrix at the next time step:

$$
\hat{x_{t}} =
A_{t-1} \cdot x_{t-1}+ B_{t-1} \cdot u_{t-1} = f(x_{t-1}, u_{t-1})
$$

$$
\hat{P_{t}} =
J_{t-1} \cdot \hat{P}_{t-1} \cdot J_{t-1}^T + Q
$$

The update step is used to update the state of the robot based on the measurements.

$$
measurement\; residual_{t} =
z_{t} - H_{t} \cdot x_{t}
$$

After calculating the Kalman Gain $K_{t}$, the state of the robot and the process covariance matrix are updated as follows:

$$
x_{t} = \hat{x_{t}} + K_{t} \cdot measurement\;residual_{t}
$$
     
$$
P_{t} = (I - K_{t} \cdot H_{t}) \cdot \hat{P_{t}}
$$

Below is shown an image and a video of the robot following the optimal path with camera obstruction. The red line is the optimal path, the blue line is the actual path followed by the robot and the green line is the estimated path of the robot.

<div>
<img src="Images/4_kalman.png" width="400"/>
</div>

<div>
<img src="Images/kalman.gif" width="400"/>
</div>

The Kalman filter allows to estimate well the position of the robot and to follow the optimal path even when the camera is obstructed.

In [ ]:
def getJacobainA(yaw, deltak, v):
    """
    Calculates and returns the Jacobian matrix for the Extended Kalman Filter, considering a linearized dynamics model
    """
    jacobianA = np.array([
        [1.0, 0.0, -deltak * v * math.sin(yaw), deltak * math.cos(yaw), 0.0],
        [0.0, 1.0, deltak * v * math.cos(yaw), deltak * math.sin(yaw), 0.0],
        [0.0, 0.0, 1.0, 0.0, deltak],
        [0.0, 0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 1.0]
        ])

    return jacobianA

def getB(yaw, deltak):
    """
    Calculates and returns the B matrix, which is time variant
    """
    B = np.array([  [np.cos(yaw)*deltak, 0],
                    [np.sin(yaw)*deltak, 0],
                    [0, deltak],
                    [1, 0],
                    [0, 1]])
    return B

def ekf1(z_k_observation_vector, state_estimate_k_minus_1, 
        control_vector_k_minus_1, P_k_minus_1, dk, camera_is_obstructed):
    """
    Extended Kalman Filter. Implements the Extended Kalman Filter for the Thymio robot.
         
    INPUT
        z_k_observation_vector      The observation from the Camera and from the Odometry

        state_estimate_k_minus_1    State estimate at time k-1
            
        control_vector_k_minus_1    The control vector applied at time k-1

        P_k_minus_1                 The state covariance matrix estimate at time k-1

        dk                          Sampling time in seconds

        camera_obstructed           Boolean value indicating whether the camera is obstructed or not
             
    OUTPUT
        state_estimate_k            State estimate at time k  
        
        P_k                         State covariance_estimate for time k
        
    """
    

 
    # Process noise
    process_noise_v_k_minus_1 = np.array([0,0,0,0,0])


    # Sensor noise. 
    sensor_noise_w_k = np.array([0,0,0,0,0])

    A_k_minus_1 = np.array([[1.0,  0,   0, 0, 0],
                            [  0,1.0,   0, 0, 0],
                            [  0,  0, 1.0, 0, 0],
                            [  0,  0, 0, 0, 0],
                            [  0,  0, 0, 0, 0]])

    # State model noise covariance matrix Q_k
    var_x = 0.35289
    var_y = 0.35289
    var_theta = 0.0013
    var_v = 0.00339
    var_w = 0.00157

    Q_k = np.array([[var_x,   0.0,     0.0,    0.0, 0.0],
                    [0.0,     var_y,   0.0,    0.0, 0.0],
                    [0.0,     0.0,     var_theta, 0.0, 0.0],
                    [0.0,     0.0,     0.0,    var_v, 0.0],
                    [0.0,     0.0,     0.0,    0.0,   var_w]])
                    
    # Measurement matrix H_k
    H_k = np.array([[1.0,  0.0, 0.0, 0.0, 0.0],
                    [0.0,  1.0, 0.0, 0.0, 0.0],
                    [0.0,  0.0, 1.0, 0.0, 0.0],
                    [0.0, 0.0, 0.0, 1.0, 0.0],
                    [0.0, 0.0, 0.0, 0.0, 1.0]])
                            
    # Sensor measurement noise covariance matrix R_k
    r_x = 0.003
    r_y = 0.003
    r_theta = 0.001
    r_v = 0.00339
    r_w = 0.00157
    R_k = np.array([[r_x,   0,    0, 0.0, 0.0],
                    [  0, r_y,    0, 0.0, 0.0],
                    [  0,    0, r_theta, 0.0, 0.0],
                    [0.0,  0.0, 0.0, r_v, 0.0],
                    [0.0, 0.0, 0.0, 0.0, r_w]]) 

    r_obs = np.inf
    R_k_obstr = np.array([[r_obs,   0,    0, 0.0, 0.0],
                    [  0, r_obs,    0, 0.0, 0.0],
                    [  0,    0, r_obs, 0.0, 0.0],
                    [0.0,  0.0, 0.0, r_v, 0.0],
                    [0.0, 0.0, 0.0, 0.0, r_w]])

    # PREDICT
    # Predict the state estimate at time k based on the state 
    # estimate at time k-1 and the control input applied at time k-1.
    state_estimate_k = A_k_minus_1 @ state_estimate_k_minus_1 + (
            (getB(state_estimate_k_minus_1[2],dk)) @ control_vector_k_minus_1 + 
            process_noise_v_k_minus_1)
             
    
    
             
    # Predict the state covariance estimate based on the previous
    # covariance and some noise

    jacobianA = getJacobainA(state_estimate_k_minus_1[2], dk, control_vector_k_minus_1[0])
    

    P_k = jacobianA @ P_k_minus_1 @ jacobianA.T + (Q_k)
         
    # UPDATE
    # Calculate the difference between the actual sensor measurements
    # at time k minus what the measurement model predicted 
    # the sensor measurements would be for the current timestep k.
    measurement_residual_y_k = z_k_observation_vector - (
            (H_k @ state_estimate_k) + (
            sensor_noise_w_k))

    state_estimate_k[2] = state_estimate_k[2] % (2*np.pi)
    
    # Calculate the measurement residual covariance  
    if camera_is_obstructed:

        S_k = H_k @ P_k @ H_k.T + R_k_obstr
    else:
        S_k = H_k @ P_k @ H_k.T + R_k


    
    
         
    # Calculate the near-optimal Kalman gain
    # We use pseudoinverse since some of the matrices might be
    # non-square or singular.
        
    
    
    
    K_k = P_k @ H_k.T @ np.linalg.pinv(S_k)
        
    # Calculate an updated state estimate for time k
    state_estimate_k = state_estimate_k + (K_k @ measurement_residual_y_k)


     
    # Update the state covariance estimate for time k
    P_k = P_k - (K_k @ H_k @ P_k)
      

    return state_estimate_k, P_k

<a id="3.4.5Conversion-functions-from-Thymio-speed-to-[rad/s]-and-viceversa"></a>
#### 3.4.5Conversion functions from Thymio speed to [rad/s] and viceversa

The speed of the motors used as control input for the Kalman filter is given in [mm/s]. The conversion from Thymio units to rad/s is done by calculating the conversion ratio. The latter is found from the following formula:

$ \text{speed in Rad/s} = \frac{{\text{left\_revolutions} \cdot 2 \pi}}{{\text{revolutions\_time}}}$

$\text{conversion factor} = \frac{\text{speed in Thymio units}}{\text{speed in Rad/s}}$

The above value is computed by making thymio rotate first in clockwise direction (by setting right velocity to 100 and left to 0) and after in anticlockwise direction (by setting left velocity to 100 and right to 0). The number of rotations is computed with a protractor. After doing different experiments, averages are computed for left and right wheels. 
Experimentally, one can prove that this conversion ratio is changing depending on the velocity, so the ralation is not linear. During the project, the speed is set to 100 Thymio units, hence the conversion ratio is calculated for this value. 


<div>
<img src="Images/R_estimation.jpeg" width="400"/>
</div>

Thymio with protractor for measurements.

<div>
<img src="Images/conversion_calculations.png" width="1000"/>
</div>

Table for conversion ratio calculation.

In [ ]:
# Find the conversion factor from the robot speed [linear and angular velocity in absolute reference] to wheel speed in Thymio units:
def convert_to_motor_speed(v, w, R, L, conv_ratio_right, conv_ratio_left): 
      
    # converting the linear and angular velocities to motor speeds
    vr = (2*v + w*L)/(2*R) # right motor speed
    vl = (2*v - w*L)/(2*R) # left motor speed
    
    # converting the motor speeds to Thymio units
    vr_thymio = vr * conv_ratio_right
    vl_thymio = vl * conv_ratio_left
    
    return vr_thymio, vl_thymio


In [ ]:
# Find the conversion factor from wheel speed in Thymio units to the robot speed [linear and angular velocity in absolute reference] :
def convert_to_absolute_speed(vr_thymio, vl_thymio, R, L, conv_ratio_right, conv_ratio_left): 

    # convert Thymio speeds to rad/s velocities
    vr = vr_thymio / conv_ratio_right
    vl = vl_thymio / conv_ratio_left
    
    # convert rad/s velocities to linear and angular velocities
    v = R*(vr + vl)/2   # linear velocity
    w = R*(vr - vl)/L   # angular velocity
    

    return v, w


<a id="3.4.6Transformation-of-coordinates-used-by-the-A*-algorithm,-Vision-and-Motion-Control"></a>
#### 3.4.6Transformation of coordinates used by the A* algorithm, Vision and Motion Control

The A* algorithm, Vision and Motion Control use different coordinate systems. In order to make them work together, it's necessary to transform the coordinates from one system to another. Below is shown a representation of the different coordinate systems used.


<div>
<img src="Images/reference.jpg" width="400"/>
</div>

Legend:
Blue: A* algorithm
Red: Vision
Green: Motion Control

In [ ]:
def img_to_map_coordinates(coord_tuple, image_height):
    '''
    Takes a tuple as input in the img coordinates
    and returns the tuple in the map (control) coordinates
    '''
    x, y, z = coord_tuple
    new_y = image_height - y
    z = 2 * math.pi - z
    return (x, new_y, z)


def grid_to_map_coordinates(coordinates_list, image_height):
    """
    Takes as input a list of tuples in the grid coordinates
    and returns list of tuple in the map (control) coordinates

    """
    transformed_coordinates = []

    for x, y in coordinates_list:
        new_x = y  # Inversion of x axis
        new_y = image_height - x
        transformed_coordinates.append((new_x, new_y))

    return transformed_coordinates

<a id="3.5Local-naviaton"></a>
### 3.5Local naviaton

Simultaneously following the optimal path, the robot must be able to avoid obstacles placed in its way. To detect added obstacles, we use the robot's proximity sensors, enabling it to avoid them while staying as close as possible to the optimal path.

The obstacle avoidance process comprises three main stages:

- Detection of Obstacles: Achieved through the infrared sensor.
- Avoidance of Obstacles: Changing direction to no longer detect the obstacle.
- Passing the Obstacle: Ensuring continuation along the intended path post-avoidance.

**Detection of Obstacles:**

The initial condition for obstacle avoidance is to retrieve data from the proximity sensors. To apply this condition, all stationary obstacles are printed in 2D on the map, while mobile obstacles are in 3D. In this way, we ensure that we trigger the obstacle avoidance program at the right moment.


**Avoidance of Obstacles:**

Upon detecting an obstacle, the robot reacts as if repelled by it. This response is implemented by assigning varying weights to each proximity sensor. These weights are chosen to establish a certain threshold between the robot and the obstacle, enabling in a second step, the robot to reach the global path without colliding with the obstacle while attempting to reach it.


**Passing the Obstacle:**

To resume reaching the next point on the computed optimal path, we continue running the control that was stopped during the obstacle avoidance. Since it doesn't perceive the fixed obstacle during this very brief period, rerunning the control immediately afterward was the established trade-off: to avoid the obstacle while staying as close as possible to the global path. This minor drawback appears only in some tricky positions of the obstacle, where the robot could drive onto it. In such specific cases, it might be better to recompute the optimal path. In our scenario, the vision was too slow to create a smooth movement for avoidance and required the robot to stop. This is why we decided to stick with this method of obstacle navigation.

The Thymio is capable of detecting objects on both sides. It will move in the direction where no obstacles are detected, reducing the time away from the global path. As the chosen shape for the local obstacle is a cylinder, even if we place the obstacle in front of the robot, there will always be a side detected before the other, allowing the robot to avoid it but in a more agressive way. 

Below is shown a video of the robot following the optimal path.

<div>
<img src="Images/local_avoidance.gif" width="400"/>
</div>

In [ ]:
viewobsTRH=1000

async def obstacle(node, client)->bool:
    """
    Return true when proximity sensors detecte something
    """
    #delay to get proximity sensor values
    for i in range(3):
        list_prox=list(node["prox.horizontal"])
        await client.sleep(0.01)
    if((list_prox[0]> viewobsTRH) | (list_prox[1]> viewobsTRH) | (list_prox[3]> viewobsTRH) | (list_prox[4]> viewobsTRH)):
        return True
    else:
        return False



obst_gain = [2, 3, -1, -3, -2]  ## weights repulsive field
nbr_prox_sensor = 5

async def obstacle_avoidance(node, client):
    """
    First it avoides the obstacle, second stores the speeds assigned to the left and right wheels and 
    invertes them after 2 seconds to achieve the necessary reorientation.
    """
    while await obstacle(node, client):
        # normal speed
        speed_left = 100
        speed_right = 100
        #delay to get proximity values
        for i in range(3):
            list_prox=list(node["prox.horizontal"])
            await client.sleep(0.01) 
        # repulsive filed  local obstacles
        for i in range(nbr_prox_sensor):
            speed_left += list_prox[i] * obst_gain[i] // 100
            speed_right += + list_prox[i] * obst_gain[4 - i] // 100
        # motor control
        motor_left_target = int(speed_left)
        motor_right_target = int(speed_right)
        
        await node.set_variables(motors(motor_left_target, motor_right_target))
        # delay between updates
        await client.sleep(0.25)
    await node.set_variables(motors(speed0,speed0))
    await client.sleep(2)
    print('wait 2 sec')

<a id="3.6Control-of-the-robot"></a>
### 3.6Control of the robot

<div>
<img src="Images/control.jpg" width="400"/>
</div>

In order to control the robot, a proportional controller on angular error of the robot is used. The angular error is computed as the difference between the orientation of the next point on the optimal path and the current orientation of the robot. The angular error is computed as follows:

$$
angle\; correction = 
atan(\delta\;y / \delta\;x) - \gamma
$$

where $\delta\;y$ and $\delta\;x$ are the differences between the y and x coordinates of the next point on the optimal path and the current position of the robot, and $\gamma$ is the current orientation of the robot.

The desired angular velocity is then computed as follows:

$$
\omega_{desired} =
k_{p} \cdot angle\; correction

$$


Below is shown an image and a video of the robot following the optimal path. The red line is the optimal path, the blue line is the actual path followed by the robot.

<div>
<img src="Images/1_only control.png" width="400"/>
</div>

<div>
<img src="Images/control.gif" width="400"/>
</div>

It's possible to notice that the robot followes the path correctly, even if there is some overshooting in sharp turns (3 cm.). The explanation of this can be found in the section 4.

<a id="3.7Kidnapping-of-the-robot"></a>
### 3.7Kidnapping of the robot

If the robot is kidnapped, it will be detected using the acceleometer of the thymio. When a kidnapping is detected, the camera continues to scan the environment and tries to detect the robot and the goal on the same frame. This condition is verified when the robot is put somewhere else after kidnapping. Once the robot is put somewhere else, the new global path is computed and the program runs as usual.

In [ ]:
if node.var['acc'][2] <= 13:  # If the robot is lifted up, it is considered as kidnapped

    await node.set_variables(motors(0, 0, verbose=False)) # Stop the robot wheels   
    
    print('Robot kidnapped !')

    ####################### START AND GOAL ############################
    # Capture the initial frame for initial robot and goal poses

    goal_pos_mm = (0, 0)
    state_vision_mm = (0, 0, 0)

    while (goal_pos_mm == (0, 0) or state_vision_mm == (0, 0, 0)): # Break the loop only if the goal and the robot are detected on the same frame
        
        ret, frame = cap.read()

        # get the goal position in mm
        goal_pos_mm = get_goal_pos(frame, map_size, corner_position)

        # Get the initial pose of the robot in mm
        state_vision_mm = np.array([0, 0, 0])
        state_vision_mm = get_robot_pose(frame, map_size, corner_position, state_vision)

# Optimal path is computed, and the program runs as usual

<a id="4.Further-improvements"></a>
## 4.Further improvements

Regarding path following, we used a proportional control based on the angle of the robot. This may leads to overshootings in case of short and sharp turns. In order to try to remove the overshootings, we tried to use a PID controller. However, the improvements were not significative.
The reason for this problem is related to the choice of the global navigation algorithm (A*) for computing the optimal path. Indeed it is based on the discretization of the map and gives a temporary goal for robot path and the distance between each goal is related to the cell of the occupancy grid. A smaller cell size can give very realistic object shape, but it gives a path which could be difficult to follow in terms of control. Indeed, as temporary goals are updated frequently, the error used for control (angle of the temporary goal - the robot's acctual angle), changes continuously. If apply a D controller with our reference goal updated and the error changing constantly, we receive sharp angles, and the corrections become non significant.

<a id="Conclusions"></a>
## Conclusions

We realized a system which works well in a predefined environment accepting some assumptions. Starting from images captured from a camera, an occupancy grid is created and instantanious position of the robot is read, thanks to computer vision alogorithms relying on the CV2 library. A* algorithm is deployed in order to compute the optimal path from the initial position of the robot to a predefined goal. In order to localize the robot a Bayesian filter is applied (in particular Extended Kalman filter) to fuse the information coming from odometry and vision considering the motion model of the robot. To control the robot during the navigation a proportional controller is implemented.
As a result our system is able to follow in a appropriate way the optimal computed path avoiding fixed obstacle without using the sensors to detect them and also avoiding mobile obstacles. The system is also able to recover from kidnapping. In addition, the Kalman Filter demonstrated to be a very efficient tool to fuse information coming from different sensors and to estimate the state of the robot, allowing to follow the optimal path even when the camera is obstructed.

We have achieved a robust system, able to meet requirements. In addition our team honed teamwork abilities, fostering effective collaboration. The experience cultivated invaluable soft skills, enhancing communication, problem-solving, and adaptability. These competencies are not only pivotal for successful project completion but also serve as enduring assets for our professional journey ahead.

<a id="References"></a>
## References

https://automaticaddison.com/how-to-derive-the-state-space-model-for-a-mobile-robot/

Lessons from "Multivariable control" course, EPFL, 2023/2024

Lessons from "Measurements" course, Politecnico di Milano, 2020/2021

Lessons and Excercise sessions from "Basics of Mobile Robotics" course, EPFL, 2023/2024